In [48]:
import datetime

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from elarb import datasets, util

## Load data

In [59]:
#df = datasets.download_nordpool('2022-10-09', '2022-10-11')
#df = df[df.HourDK >= '2022-10-09 12:00:00'].reset_index()
#df.to_csv('../data/example.csv', index=False)
df = pd.read_csv('../data/example.csv')
spot_dkk = df.SpotPriceDKK.values / 1000.0
spot_dkk

array([0.44604001, 0.37189002, 0.40722   , 0.38260001, 0.98460999,
       1.36169995, 1.47892004, 1.92443994, 1.07883997, 0.70509998,
       0.68940002, 0.45995001, 0.51596002, 0.40185998, 0.07274   ,
       0.06716   , 0.07088   , 0.08412   , 0.95025   , 1.69030005,
       1.98036999, 1.61532996, 1.22744995, 1.18163   , 0.99264001,
       0.97494   , 0.97560999, 1.03659998, 1.18156006, 1.56623999,
       1.84315002, 2.05832007, 1.33432996, 1.17464002, 1.15581995,
       0.89945001])

## Optimal policy

```
           ┌───────┐       
    ┌─x1──▶│ Grid  │──┐    
    │      └───────┘  │    
┌───────┐      ▲      │    
│ Solar │      │x3    │x4    
└───────┘      │      │    
    │      ┌───────┐  │    
    └─x2──▶│Battery│◀─┘    
           └───────┘       
```

### Parameters

In [92]:
n = len(spot_dkk)
BIG_NUMBER = 100

# 1 inverter: Deye SUN-12K-SG04LP3-EU

# grid
p_grid = cp.Parameter(n, nonneg=True)
p_grid.value = spot_dkk
d_grid = cp.Parameter(n, nonneg=True)
d_grid.value = np.zeros(n) + BIG_NUMBER

# 1 battery: Powerwall LBATTS Powerwall, 8.8 kwh
bat_cap = 8.8
bat_discharge = 11.52
bat_loss = 0.03

p_bat = cp.Parameter(n, nonneg=True)
p_bat.value = np.zeros(n)

# 33 solar panels: 470Wp JinKO TigerNeo N-Type 60HL4 BF, 1.57 kWh / time
solar_kWh = 1.57

p_solar = cp.Parameter(n, nonneg=True)
p_solar.value = np.zeros(n) + 0.16
s_solar = cp.Parameter(n, nonneg=True)
s_solar.value = np.zeros(n) + solar_kWh

### Variables

In [74]:
x1 = cp.Variable(n, nonneg=True)
x2 = cp.Variable(n, nonneg=True)
x3 = cp.Variable(n, nonneg=True)
x4 = cp.Variable(n, nonneg=True)

### Objective function

In [79]:
contrib_x1 = p_grid@x1 + p_grid - p_solar@x1
contrib_x2 = p_bat@x2 - p_solar@x2
contrib_x3 = (1-)p_grid@x3 - p_bat@x3
contrib_x4 = p_bat@x4 - p_grid@x4
objective = cp.Maximize(cp.sum(contrib_x1 + contrib_x2 + contrib_x3 + contrib_x4))

### Constraints

In [78]:
constraints = [
    x1 + x3 <= d_grid,
#    x2 + x4 <= d_bat,
    x1 + x2 <= s_solar,
#    x3 <= s_bat,
    x4 <= s_grid,
    
    
]
constraints
constraint2 = 
constraint3 = x3 <= 42
constraint4 = x4 <= 42
constraints = [constraint1, constraint2, constraint3, constraint4]

### Solve problem

In [87]:
prob = cp.Problem(objective, constraints)
result_arbitrage = prob.solve()
result_solaronly = s_solar@p_grid

result_arbitrage, result_solaronly.value

(98722.40227484348, 55.48703411051)

In [88]:
#print(result)
# The optimal value for x is stored in `x.value`.
#print(x.value.round(2))

# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
#print(np.round(constraints[0].dual_value, 2))


#plt.bar(c, x.value)
#plt.show()

In [36]:
cp.cumsum(c).value

array([ 0.,  1.,  3.,  6., 10., 15., 21.])

In [34]:
cp.max(cp.cumsum(c)).value

array(21.)